# Final Distribution

In [1]:
from config import PROJECT_ID, INITIAL_TS, SNAPSHOT_TS, \
    GENESIS_AND_ETH2_TYPE, GENESIS_AND_ETH2_DATASET_NAME, GENESIS_AND_ETH2_DISTRIBUTION_TABLE_NAME, \
    GAS_ANALYSIS_DATASET_NAME, GAS_ANALYSIS_DISTRIBUTION_TABLE_NAME, GAS_ANALYSIS_TYPE, \
    ETH_ANALYSIS_TYPE, ETH_ANALYSIS_DATASET_NAME, ETH_ANALYSIS_DISTRIBUTION_TABLE_NAME, \
    ERC721_ANALYSIS_TYPE, ERC721_ANALYSIS_DATASET_NAME, ERC721_ANALYSIS_DISTRIBUTION_TABLE_NAME, \
    ERC20_ANALYSIS_TYPE, ERC20_ANALYSIS_DATASET_NAME, ERC20_ANALYSIS_DISTRIBUTION_TABLE_NAME, \
    ERC20_TOP_TYPE, ERC20_TOP_DISTRIBUTION_TABLE_NAME, \
    FINAL_DATASET_NAME, FINAL_DISTRIBUTION_VIEW_NAME, CONTRACT_ADDRESSES_TABLE
from utils import create_view, create_table, drop_table

DROP_TABLES = True
CREATE_TABLES = True
DROP_VIEW = True
CREATE_VIEW = True

## Create Contracts Table

In [2]:
query_1 = f'''
    SELECT
        to_address as address
    FROM `bigquery-public-data.crypto_ethereum.traces`
    WHERE block_timestamp >= '{INITIAL_TS}'
      AND block_timestamp <= '{SNAPSHOT_TS}'
      AND trace_type ='create'
      AND status = 1
'''

if DROP_TABLES:
    drop_table(CONTRACT_ADDRESSES_TABLE, dataset_name=FINAL_DATASET_NAME)
if CREATE_TABLES:
    create_table(query_1, CONTRACT_ADDRESSES_TABLE, dataset_name=FINAL_DATASET_NAME)

Table final:contract_addresses has not been deleted.
404 DELETE https://bigquery.googleapis.com/bigquery/v2/projects/cosmic-keep-223223/datasets/final/tables/contract_addresses?prettyPrint=false: Not found: Table cosmic-keep-223223:final.contract_addresses
Table final:contract_addresses has been created and filled 38527292 rows.


## Create View

In [4]:
query_1 = f'''
    SELECT
        type,
        subtype,
        address,
        grade
    FROM (
        SELECT
            '{GENESIS_AND_ETH2_TYPE}' AS type,
            subtype,
            address,
            grade
        FROM `{PROJECT_ID}.{GENESIS_AND_ETH2_DATASET_NAME}.{GENESIS_AND_ETH2_DISTRIBUTION_TABLE_NAME}`
        WHERE grade IS NOT NULL

        UNION ALL

        SELECT
            '{GAS_ANALYSIS_TYPE}' AS type,
            subtype,
            address,
            grade
        FROM `{PROJECT_ID}.{GAS_ANALYSIS_DATASET_NAME}.{GAS_ANALYSIS_DISTRIBUTION_TABLE_NAME}`
        WHERE grade IS NOT NULL

        UNION ALL

        SELECT
            '{ETH_ANALYSIS_TYPE}' AS type,
            subtype,
            address,
            grade
        FROM `{PROJECT_ID}.{ETH_ANALYSIS_DATASET_NAME}.{ETH_ANALYSIS_DISTRIBUTION_TABLE_NAME}`
        WHERE grade IS NOT NULL

        UNION ALL

        SELECT
            '{ERC721_ANALYSIS_TYPE}' AS type,
            '' AS subtype,
            address,
            grade
        FROM `{PROJECT_ID}.{ERC721_ANALYSIS_DATASET_NAME}.{ERC721_ANALYSIS_DISTRIBUTION_TABLE_NAME}`
        WHERE grade IS NOT NULL

        UNION ALL

        SELECT
            '{ERC20_ANALYSIS_TYPE}' AS type,
            '' AS subtype,
            address,
            grade
        FROM `{PROJECT_ID}.{ERC20_ANALYSIS_DATASET_NAME}.{ERC20_ANALYSIS_DISTRIBUTION_TABLE_NAME}`
        WHERE grade IS NOT NULL

        UNION ALL

        SELECT
            '{ERC20_TOP_TYPE}' AS type,
            '' AS subtype,
            address,
            grade
        FROM `{PROJECT_ID}.{ERC20_ANALYSIS_DATASET_NAME}.{ERC20_TOP_DISTRIBUTION_TABLE_NAME}`
        WHERE grade IS NOT NULL)
    WHERE address NOT IN (SELECT address FROM `{PROJECT_ID}.{FINAL_DATASET_NAME}.{CONTRACT_ADDRESSES_TABLE}`)
'''

if DROP_VIEW:
    drop_table(FINAL_DISTRIBUTION_VIEW_NAME, dataset_name=FINAL_DATASET_NAME)

if CREATE_VIEW:
    create_view(query_1, FINAL_DISTRIBUTION_VIEW_NAME)

Table final:final_distribution has not been deleted.
404 DELETE https://bigquery.googleapis.com/bigquery/v2/projects/cosmic-keep-223223/datasets/final/tables/final_distribution?prettyPrint=false: Not found: Table cosmic-keep-223223:final.final_distribution
View VIEW:cosmic-keep-223223.final.final_distribution has been created.
